In [48]:
import pandas as pd
import os
from pathlib import Path

In [49]:
# 데이터 경로
data_dir = Path("/Users/parkjuyong/Desktop/4-1/CareerRoute/assets/tiobe")

# 결과 저장 경로
output_dir = Path("/Users/parkjuyong/Desktop/4-1/CareerRoute/ml/trend_tech_ml/data")
output_dir.mkdir(parents=True, exist_ok=True) # 파일 없으면 자동으로 생성해줌

# 최종 파일 경로
output_file = output_dir / "integrated_tiobe_data.csv"

In [50]:
# 매핑 언어 딕셔너리로 정의
language_mapping = {
    'python': 'python',
    'java': 'java',
    'c++': 'cpp',
    'c#': 'csharp',
    'c': 'c',
    'javascript': 'javascript',
    'sql': 'sql',
    'r': 'r',
    'visual-basic': 'visual_basic',
    'perl': 'perl'
}

all_languages = list(language_mapping.keys()) # key값들로 리스트에 저장

In [51]:
# 테스트로 하나씩 로드
def load_csv(filepath: Path) -> pd.DataFrame:
    df=pd.read_csv(filepath)
    df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m') # 일까지 포함하면 시계열의 간격이 불규칙해짐
    return df

In [52]:
# 테스트
test_df = load_csv(data_dir / "python.csv")
test_df.head()

,date,percent
0,2001-06,1.25
1,2001-07,1.13
2,2001-08,1.20
3,2001-09,1.17
4,2001-10,1.28


In [53]:
# 모든 언어 csv로드
all_language = {} # 빈 딕셔너리

# 모든 파일 돌아가면서 컬럼을 위의 언어 매핑 방식으로 다시 정의 (c++의 이름으로 저장된 파일의 언어는 cpp로 컬럼 정의)
for lang_file, lang_col in language_mapping.items():
    csv_path = data_dir / f"{lang_file}.csv"

    if not csv_path.exists():
        print(f"{csv_path.name} 의 파일은 없습니다. 스킵합니다.")
        continue

    df = load_csv(csv_path)
    # percent컬럼을 언어 이름으로 바꿈(나중에 시계열 데이터로 한번에 나타낼때 컬럼이 날짜와 언어가 됨)
    df = df.rename(columns={'percent': lang_col}) 

    all_language[lang_col] = df
    print(f"{lang_file} 로드 완료 ({len(df)} rows)")
    

python 로드 완료 (291 rows)
java 로드 완료 (291 rows)
c++ 로드 완료 (291 rows)
c# 로드 완료 (291 rows)
c 로드 완료 (291 rows)
javascript 로드 완료 (291 rows)
sql 로드 완료 (130 rows)
r 로드 완료 (222 rows)
visual-basic 로드 완료 (186 rows)
perl 로드 완료 (291 rows)


In [54]:
all_language.keys()

dict_keys(['python', 'java', 'cpp', 'csharp', 'c', 'javascript', 'sql', 'r', 'visual_basic', 'perl'])

In [55]:
# 언어별 df를 하나의 df로 머지함
merge_df = None

for lang, df in all_language.items():
    if merge_df is None:
        merge_df = df
    else:
        merge_df = pd.merge( # 특정 컬럼 기준으로 가로로 합침
            merge_df,
            df,
            on='date', # 날짜 기준 머지(기준 컬럼)
            how='outer' # 합집합 개념
        )

In [56]:
merge_df.head()

,date,python,java,cpp,csharp,c,javascript,sql,r,visual_basic,perl
0,2001-06,1.25,26.49,14.20,0.38,20.24,1.55,2.96,NaN,NaN,6.60
1,2001-07,1.13,25.03,16.11,0.43,20.77,1.72,2.77,NaN,NaN,6.78
2,2001-08,1.20,24.66,16.12,0.38,20.75,1.66,2.38,NaN,NaN,7.07
3,2001-09,1.17,24.82,15.85,0.39,20.77,1.63,2.36,NaN,NaN,7.31
4,2001-10,1.28,25.68,16.10,0.42,19.75,1.51,2.24,NaN,NaN,7.38


In [57]:
# 날짜 정렬
merge_df = merge_df.sort_values('date').reset_index(drop=True) # date기준 오름차순 정렬 후 인덱스 재정의
merge_df.head()

,date,python,java,cpp,csharp,c,javascript,sql,r,visual_basic,perl
0,2001-06,1.25,26.49,14.20,0.38,20.24,1.55,2.96,NaN,NaN,6.60
1,2001-07,1.13,25.03,16.11,0.43,20.77,1.72,2.77,NaN,NaN,6.78
2,2001-08,1.20,24.66,16.12,0.38,20.75,1.66,2.38,NaN,NaN,7.07
3,2001-09,1.17,24.82,15.85,0.39,20.77,1.63,2.36,NaN,NaN,7.31
4,2001-10,1.28,25.68,16.10,0.42,19.75,1.51,2.24,NaN,NaN,7.38


In [58]:
# 겹치는 기간으로 필터링
START_DATE = "2010-09"

filtered_df = merge_df[merge_df['date'] >= START_DATE]
filtered_df = filtered_df.dropna()

In [59]:
filtered_df.to_csv(output_file, index=False)

print("저장 완료")
print(f"Rows: {len(filtered_df)}")
print(f"Date range: {filtered_df['date'].iloc[0]} ~ {filtered_df['date'].iloc[-1]}")
print("Languages:", list(filtered_df.columns[1:]))

저장 완료
Rows: 1117
Date range: 2018-02 ~ 2025-12
Languages: ['python', 'java', 'cpp', 'csharp', 'c', 'javascript', 'sql', 'r', 'visual_basic', 'perl']


In [60]:
filtered_df.tail()

,date,python,java,cpp,csharp,c,javascript,sql,r,visual_basic,perl
2582,2025-08,26.14,8.59,9.18,5.52,9.03,3.15,1.72,1.37,2.33,2.08
2583,2025-09,25.98,8.35,8.80,6.38,8.65,3.22,1.86,1.43,2.84,2.03
2584,2025-10,24.45,8.35,8.84,6.94,9.29,3.41,1.77,1.52,3.22,1.66
2585,2025-11,23.37,8.54,8.95,7.65,9.68,3.42,1.80,1.67,3.31,1.84
2586,2025-12,23.64,8.70,8.95,7.26,10.11,2.96,2.10,1.96,2.81,1.97


# Data Scaling for LSTM Model

In [61]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import pickle

# 데이터 로드
df = pd.read_csv(output_file)
df.head()

,date,python,java,cpp,csharp,c,javascript,sql,r,visual_basic,perl
0,2018-02,5.17,14.99,5.73,4.45,11.86,3.16,2.36,2.09,2.18,1.76
1,2018-03,5.87,14.94,6.45,5.07,12.76,3.92,2.69,1.13,1.74,2.23
2,2018-04,5.80,15.78,7.22,5.27,13.59,3.49,2.65,1.81,1.80,1.53
3,2018-05,5.19,16.38,7.67,4.40,14.00,2.92,1.99,1.18,0.91,0.91
4,2018-06,5.76,15.37,8.34,4.31,14.94,2.50,2.34,1.45,1.15,1.15


In [62]:
# 날짜는 스케일링 x -> 분리하여 저장
dates = df['date'].values
numeric_columns = df.columns[1:].tolist()
data = df[numeric_columns].values

In [63]:
# 0-1 범위로 스케일링하는 객체 생성
scaler = MinMaxScaler(feature_range=(0, 1))

# 각 컬럼의 최소값/최대값을 학습 후 실제 데이터를 변환
scaled_data = scaler.fit_transform(data)

print(f"Original data shape: {data.shape}")
print(f"Scaled data shape: {scaled_data.shape}")
print(f"\nOriginal data range:")
print(f"  Min: {data.min(axis=0)}")
print(f"  Max: {data.max(axis=0)}")
print(f"\nScaled data range:")
print(f"  Min: {scaled_data.min(axis=0)}")
print(f"  Max: {scaled_data.max(axis=0)}")

Original data shape: (1117, 10)
Scaled data shape: (1117, 10)

Original data range:
  Min: [5.17 7.87 5.57 2.85 8.38 1.78 1.39 0.72 0.6  0.44]
  Max: [26.98 17.8  13.94  8.21 17.38  4.61  2.87  2.79  5.86  2.23]

Scaled data range:
  Min: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  Max: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [64]:
# 스케일된 데이터를 다시 저장
scaled_df = pd.DataFrame(scaled_data, columns=numeric_columns)
scaled_df.insert(0, 'date', dates)

# csv로 저장
scaled_output_file = output_dir / "scaled_tiobe_data.csv"
scaled_df.to_csv(scaled_output_file, index=False)
print(f"Scaled data saved to: {scaled_output_file}")

# 나중에 예측값 역변환을 위해 스케일러 저장
scaler_file = output_dir / "scaler.pkl"
with open(scaler_file, 'wb') as f:
    pickle.dump(scaler, f)
print(f"Scaler saved to: {scaler_file}")

scaled_df.head()

Scaled data saved to: /Users/parkjuyong/Desktop/4-1/CareerRoute/ml/trend_tech_ml/data/scaled_tiobe_data.csv
Scaler saved to: /Users/parkjuyong/Desktop/4-1/CareerRoute/ml/trend_tech_ml/data/scaler.pkl


,date,python,java,cpp,csharp,c,javascript,sql,r,visual_basic,perl
0,2018-02,0.000000,0.717019,0.019116,0.298507,0.386667,0.487633,0.655405,0.661836,0.300380,0.737430
1,2018-03,0.032095,0.711984,0.105137,0.414179,0.486667,0.756184,0.878378,0.198068,0.216730,1.000000
2,2018-04,0.028886,0.796576,0.197133,0.451493,0.578889,0.604240,0.851351,0.526570,0.228137,0.608939
3,2018-05,0.000917,0.856999,0.250896,0.289179,0.624444,0.402827,0.405405,0.222222,0.058935,0.262570
4,2018-06,0.027052,0.755287,0.330944,0.272388,0.728889,0.254417,0.641892,0.352657,0.104563,0.396648
